原文https://innig.net/software/ruby/closures-in-ruby

In [1]:
class MyClass
  def my_method(my_arg)
    my_arg * 2
  end
end

obj = MyClass.new
obj.my_method(3)

6

In [2]:
obj.send(:my_method, 3)

6

为什么要用`send`方法，而不用原先的点标识符呢？这个是因为在`send`方法里，你想调用的方法名编程了参数，这样就可以在代码运行的最后一刻决定调用哪个方法。这个技巧叫做**动态转发**。

In [3]:
require 'pry'

false

In [4]:
class Pry
def rf(options = {})
  defaults = {}
  attributes = [:input, :output, :commands, :print, :quiet, :execution_handler, :hooks, :custom_completions, :prompt, :memory_size, :extra_sticky_locals]
  
  attributes.each do |attribute|
    defaults[attribute] = Pry.send attribute
  end
  
  defaults.merge!(options).each do |key, value|
    send("#{key}=", value) if respond_to?("#{key}=")
  end
  true
end
end

:rf

这段代码用`send`方法把每个属性的默认值放入一张哈希表中，然后把这张哈希表和传入参数的`options`合并。最后使用`send`方法调用每个属性的写方法（如`memery_size=`)。`Kernel#respond_to?`方法检测注入`Pry#memory_size=`这样的方法是否存在，如果存在则返回`true`。这样，如果参数`options`哈希表中设置了当前属性中不存在的属性，这些属性就会被忽略。

你可以用`Module#define_method()`方法随时定义一个方法，只需要提供给一个方法名和充当方法主体的块：

In [5]:
class MyClass
  define_method :my_method do |my_arg|
    my_arg * 3
  end
end

obj = MyClass.new
obj.my_method(2)

6

`defind_method`方法在`MyClass`内部执行，因此`my_method`定义为`MyClass`的实例方法。这种在运行时定义方法的技术称为动态方法

In [6]:
class Computer
  def initialize(computer_id, data_source)
    @id = computer_id
    @data_source = data_source
  end
  
  def mouse
    component :mouse
  end
  
  def cpu
    component :cpu
  end
  
  def keyboard
    component :keyboard
  end
  
  def component(name)
    info = @data_source.send "get_#{name}_info", @id
    price = @data_source.send "get_#{name}_price", @id
    result = "#{name.capitalize}: #{info} ($#{price})"
    return "* #{result}" if price >= 100
    result
  end
end

:component

In [7]:
my_computer = Computer.new(42, DS.new)
my_computer.cpu

NameError: uninitialized constant DS

In [8]:
class Computer
  def initialize(computer_id, data_source)
    @id = computer_id
    @data_source = data_source
  end
  
  def self.define_component(name)
    define_method(name) do
      info = @data_source.send "get_#{name}_info", @id
      price = @data_source.send "get_#{name}_price", @id
      result = "#{name.capitalize}: #{info} ($@{price})"
      return "* #{result}" if price >= 100
      result
    end
  end
  
  define_component :mouse
  define_component :cpu
  define_component :keyboard
end

:keyboard

注意那三个对`define_component`方法的调用执行于`Computer`的类定义域中，`Computer`类是当前的`self`。因为你是在`Computer`类上调用`define_component`方法，因此它必然是一个类方法。

可以通过内省`data_source`参数来提取所有组件的名字，去掉所有使用`define_component`定义的方法。

In [9]:
class Computer
  def initialize(computer_id, data_source)
    @id = computer_id
    @data_source = data_source
    data_source.methods.grep(/^get_(.*)_info/)
    {Computer.define_component $1}
  end
  
  def self.define_component(name)
    define_method(name) do
    end
  end
end

SyntaxError: (pry):87: syntax error, unexpected tNTH_REF, expecting =>
 {Computer.define_component $1}
                              ^
(pry):94: syntax error, unexpected end-of-input, expecting keyword_end

In [10]:
class Lawyer; end
nick = Lawyer.new
nick.talk_simple

NoMethodError: undefined method `talk_simple' for #<Lawyer:0x17cfbe0>

In [11]:
nick.send :method_missing, :my_method

NoMethodError: undefined method `my_method' for #<Lawyer:0x17cfbe0>
Did you mean?  method

你刚刚做了Ruby解释器所做的工作。你告诉这个对象，“我试着调用你的一个名为`my_method`的fanfare”，但是你不明白我想干什么。"`BasicObject#method_missing`方法会爆出一个`NoMethodError`进行响应，这是它所有的工作。

In [12]:
class Lawyer
  def method_missing(method, *args)
    puts "You called: #{method} {#{args.join(',')}}"
    puts "{You also passed it a block}" if block_given?
  end
end

bob = Lawyer.new
bob.talk_simple('a', 'b') do
end


You called: talk_simple {a,b}
{You also passed it a block}


In [13]:
require 'hashie'


true

In [14]:
icecream = Hashie::Mash.new
icecream.flavor = "strawberry"
icecream.flavor

"strawberry"

In [15]:
module Hashie
  class Mash < Hashie::Hash
    def method_missing(method_name, *arg, &blk)
      return self.[](method_name) if key?(method_name)
      match = method_name.to_s.match(/(.*?)([?=!]?)$/)
      case match[2]
        when "="
        ...
        else
      end
    end
  end
end


SyntaxError: unexpected ...
        ...
           ^


如果被调用方法的名字存在于哈希表的主键中(比如flavor),那么`Hashie::Mash@method_missing`方法要做的只是调用`[]`方法来返回相应的值。如果名字以`=`结尾，`method_missing`方法会砍掉末尾的`=`，把余下的部分作为属性名。然后用个哈希表相应的键值对属性该熟悉你给的值。如果调用的方法名不符合上述的任何一个条件，则`method_missing`会返回一个默认名。

In [16]:
require 'ghee'
gh = Ghee.basic_auth('ceclinux','src17283950')
all_gist = gh.users('ceclinux').gists
a_gist = all_gists[20]

a_gist.url
a_gist.description
a_gist.star

Gem::ConflictError: Unable to activate ghee-0.15.23, because hashie-3.5.6 conflicts with hashie (~> 3.3.2)

详情请看`ghee 0.98`源代码

调用一个方法来改变一个对象的状态时（比如`Ghee::API::Gists#star`)，Ghee会产生一个HTTP调用来访问相应的`Github URL`。然而，如果调用的方法只是读取一个属性的值（比如`url`或者`description`），那么这个调用最终会被转发给`Ghee::ResourceProxy#method_missing`方法。然后，`method_missing`方法会把调用转发给`Ghee::ResourceProxy#subject`方法返回的对象。这究竟是个什么对象呢？

查看`ResourceProxy#subject`方法的实现，你会发现该方法也产生了一个对`Github API`的HTTP调用。具体的调用依赖于实现的`Ghee::ResourceProxy`子类（因为params不同）。例如，`Ghee::API::Gists::Proxy`会调用`https://api.gihub.com/users/nusco/gists`。`ResourceProxy#subject`方法从`github`接受JSON格式的对象，然后把它们转化为哈希表类型的对象。

深入查看，你会发现这个像哈希表类型的对象其实正是刚刚介绍过的`Hashie::Mash`对象（看Faraday Middleware和Ghee connection源码）。这意味着`my_gist.url`这样的方法调用会被首先转发给`Ghee::ResourceProxy#method_missing`方法，然后再转发给`Hashie::Mash#method_missing`方法，最终返回`url`的值。没错，连续调用两次`method_missing`方法。

In [17]:
# 用method_missing来重构

In [18]:
class Computer
  def initialize(computer_id, data_source)
    @id = computer_id
    @data_source = data_source
  end
  
  def method_missing(name)
    super if !@data_source.respond_to?("get_#{name}_info")
      info = @data_source.send("get_#{name}_info", @id)
      price = @data_source.send("get_#{name}_price", @id)
      result = "{#name.capitalize}: #{info} ($#{price})"
      return "* #{result}" if price >= 100
      result
      end
  end

:method_missing

调用像`Computer#mouse`这样的方法时，究竟会发生什么呢？ 这个调用会被传递给`method_missing`方法，在那里她会检测被封装的对象是否存在`get_mouse_info`方法。如果不存在，则这个调用会被转发回`BasicObject#method_missing`方法，并抛出一个`NoMethodError`的错误。如果数据源知道有这个部件，那么最初的调用会被转换为两个方法调用：`DS#get_mouse_info`方法和`DS#get_mouse_price`方法。这两个方法返回的值用来构造出最终返回的结果。

In [19]:
class DS
end
cmp = Computer.new(0, DS.new)
cmp.respond_to(:mouse)

ArgumentError: wrong number of arguments (given 2, expected 1)

Ruby提供了一种简洁的方法来让`respond_to？`方法感知幽灵方法。

在`respond_to?`方法中，如果该方法是一个幽灵方法，当它调用`respond_to_missing`时，会返回`true`值。,为了不让`respond_to`方法说谎，**每次覆写`method_missing`方法时，都应该同时覆写`respond_missing`方法**

In [ ]:
def respond_to_missing?(method, include_private = false)
  @data_source.respond_to?("get_#{method}_info") || super
end

`respond_to_missing?`方法的代码与`method_missing`的类似：它首先查看一个方法是不是幽灵方法。若是，则返回`true`：若不是，则调用`super`。在本例中，`super`默认的`Object#respond_to_missing？`方法，它总是返回`false`。

当引用一个不存在的常量时，Ruby会把这个常量名作为一个符号给`const_missing`方法。

幽灵方法进场会遇到问题，由于调用未定义的方法会导致调用`method_missing`方法，所以对象可能会接受错的方法调用（比如写错了方法名）。

In [4]:
class Roulette
  def method_missing(name, *args)
    person = name.to_s.capitalize
    super unless %w[Bob Frank Bill].include? person
    number = 0
    3.times do
      number = rand(10) + 1
      puts "#{number}..."
    end
    "#{person} got a #{number}"
  end
end

:method_missing

In [5]:
Object.instance_methods.grep /^d/

[:define_singleton_method, :display, :dup]

原来`Object`已经定义了一个名为`display`方法。Computer类继承自`Object`类，因此也继承了`display`方法。调用`Computer#display`方法会找到一个真正的方法，所以不会用到`method_missing`方法，你调用了一个真实的方法，而非**幽灵方法**。

继承`BasicObject`类是最简单的定义白板类的方法。不过，在某些情况下，你可能还要删除某些方法。

删除一个方法有两种途径：一种是用`Module#undef_method`方法，另一种是用`module#remove_method`方法。`module#undef_method`比较蛮横，它会删除所有（包括继承而来的）的方法；`Module#remove_method`方法比较温柔，它只删除接收者自己的方法，而保留继承来的方法。

In [ ]:
class Computer < BasicObject
end

还有一些可以改进的地方。`BasicObject`没有`respond_to?`方法（`respond_to?`方法定义在`BasicObject`的子类`Object`中）。由于没有`respond_to?`方法，前面的`respond_to_missing?`也就没有意义了，应该将它删除。

如果没有`Ruby`的动态特性，无论哪一种方法都无法实现。

使用*幽灵方法*可能带来风险。虽然可以通过一些简单的规则来规避大多数风险（比如在`method_missing`中总是；总是重新定义`method_missing？`方法），但是幽灵方法还是有可能带来令人困惑的Bug。

幽灵方法产生风险的根本原因是因为它们并非真正的方法，它们只是对方法调用的拦截。正因为如此，它们会真正的方法有所不同。比如，它们**不会出现在`Object.methods`方法返回的方法名列表中。相反，动态方法则是普通的方法，只不过它们不是用`def`定义的，而是用`define_method`定义的，它们的行为跟其他方法没有什么两样。**

不过，有时你只能选择幽灵方法。这通常是因为有非常多的方法调用，而你不知道运行时会调用什么方法。比如在Builder库的例子里，`XML`的标签是无穷的，`Builder`不可能为每一个标签产生一个动态方法，因此它只能使用`method_missing`方法进行调用拦截。

所以我们的原则是：在可以使用动态方法的时候，尽量使用动态方法：除非必须使用幽灵方法，否则不要使用它。